In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from scipy.signal import medfilt
import scipy.signal
import scipy.optimize

# ============================================================================
# CONFIGURATION
# ============================================================================

LEADS = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
MAX_TIME_SHIFT = 0.2
PERFECT_SCORE = 384


OUTLIER_LOW_THRESHOLD = -1.6
OUTLIER_HIGH_THRESHOLD = 0.85
MARKER_ARTIFACT_THRESHOLD = 0.2
MEDIAN_FILTER_SIZE = 5
IMAGE_BINARIZATION_THRESHOLD = 160
SCALING_FACTOR = 80
TAIL_CORRECTION_FACTOR = 2.0

TRAIN_DIR = '/kaggle/input/physionet-ecg-image-digitization/train/'
TEST_DIR = '/kaggle/input/physionet-ecg-image-digitization/test/'


def compute_power(label, prediction):
    finite_mask = np.isfinite(prediction)
    if not np.any(finite_mask):
        return 0, 1
    prediction = prediction.copy()
    prediction[~np.isfinite(prediction)] = 0
    noise = label - prediction
    return np.sum(label**2), np.sum(noise**2)

def compute_snr(signal, noise):
    if noise == 0: return PERFECT_SCORE
    elif signal == 0: return 0
    else: return min((signal / noise), PERFECT_SCORE)

def align_signals(label, pred, max_shift=float('inf')):
    label_arr = np.asarray(label, dtype=np.float64)
    pred_arr = np.asarray(pred, dtype=np.float64)
    
    correlation = scipy.signal.correlate(label_arr - np.mean(label_arr), 
                                        pred_arr - np.mean(pred_arr), mode='full')
    lags = scipy.signal.correlation_lags(len(label_arr), len(pred_arr), mode='full')
    valid_mask = (lags >= -max_shift) & (lags <= max_shift)
    
    best_idx = np.argmax(correlation[valid_mask])
    time_shift = lags[valid_mask][best_idx]
    
    start_pad = max(time_shift, 0)
    pred_start = max(-time_shift, 0)
    pred_end = min(len(label_arr) - time_shift, len(pred_arr))
    end_pad = max(len(label_arr) - len(pred_arr) - time_shift, 0)
    
    aligned_pred = np.concatenate((
        np.full(start_pad, np.nan),
        pred_arr[pred_start:pred_end],
        np.full(end_pad, np.nan)
    ))
    
    if np.any(np.isfinite(label_arr) & np.isfinite(aligned_pred)):
        def objective(v): return np.nansum((label_arr - (aligned_pred - v)) ** 2)
        result = scipy.optimize.minimize_scalar(objective, method='Brent')
        aligned_pred -= result.x
    
    return aligned_pred


def fit_mean_model(train_df):
    print("\n Building fallback templates...")
    mean_dict = defaultdict(list)
    
    for idx, row in tqdm(train_df.iterrows(), total=len(train_df), desc="Processing"):
        csv_path = f'{TRAIN_DIR}{row.id}/{row.id}.csv'
        if not os.path.exists(csv_path):
            continue
        
        try:
            labels = pd.read_csv(csv_path)
            for lead in labels.columns:
                values = labels[lead].dropna().values.astype(np.float32)
                if len(values) < 50:
                    continue
                
                values_norm = (values - values.mean()) / (values.std() + 1e-8)
                values_resamp = np.interp(
                    np.linspace(0, 1, 20000),
                    np.linspace(0, 1, len(values_norm)),
                    values_norm
                )
                mean_dict[lead].append(values_resamp)
        except:
            continue
    
    for lead in mean_dict.keys():
        mean_dict[lead] = np.stack(mean_dict[lead])
    
    for lead in LEADS:
        if lead not in mean_dict:
            t = np.linspace(0, 1, 20000)
            mean_dict[lead] = np.array([np.sin(2 * np.pi * t)])
    
    print(f"✓ Fallback ready for {len(mean_dict)} leads")
    return mean_dict


class MarkerFinder:
    """Marker detection adapted for 1700x2200 format"""
    
    def __init__(self):
        # Load reference images
        ref_paths = [
            f'{TRAIN_DIR}4292118763/4292118763-0001.png',
            f'{TRAIN_DIR}4289880010/4289880010-0001.png',
            f'{TRAIN_DIR}4284351157/4284351157-0001.png',
        ]
        
        ref_images = [cv2.imread(p) for p in ref_paths if os.path.exists(p)]
        if not ref_images:
            self.templates = None
            return
        
        ima = np.max(ref_images, axis=0)
        
   
        self.scale_factor_1700 = 1700 / 1652
        self.scale_factor_2200 = 2200 / 2132
        
        # Define marker positions (original coordinates)
        absolute_points = np.zeros((17, 2), dtype=int)
        for i in range(3):
            absolute_points[5 * i] = np.array([707 + 284 * i, 118])
            for j in range(1, 5):
                absolute_points[5 * i + j] = np.array([707 + 284 * i, 118 + 492 * j])
        absolute_points[15] = np.array([1535, 118])
        absolute_points[16] = np.array([1535, 118 + 492 * 4])
        
        template_positions = [None] * 17
        template_points = [None] * 17
        
        for i in range(len(absolute_points)):
            if absolute_points[i][1] < 118 + 492 * 4:
                if i % 5 == 0:
                    template_positions[i] = (absolute_points[i][0] - 87, absolute_points[i][1] - 50)
                else:
                    template_positions[i] = (absolute_points[i][0] - 37, absolute_points[i][1] - 13)
                template_points[i] = np.array([
                    absolute_points[i][0] - template_positions[i][0],
                    absolute_points[i][1] - template_positions[i][1]
                ])
        
        template_sizes = np.array([(105, 60)] * 17)
        
        self.templates = [None] * 17
        for i in range(len(template_positions)):
            if template_positions[i] is not None:
                template = ima[
                    template_positions[i][0]:template_positions[i][0] + template_sizes[i][0],
                    template_positions[i][1]:template_positions[i][1] + template_sizes[i][1]
                ]
                self.templates[i] = template
        
        self.template_positions = template_positions
        self.template_sizes = template_sizes
        self.template_points = template_points
    
    def find_markers(self, ima):
        """Find markers with scaling support for 1700x2200 images"""
        if self.templates is None or self.templates[0] is None:
            return None
        
        h, w = ima.shape[:2]
        
        # Calculate scale factors
        scale_y = h / 1652
        scale_x = w / 2132
        
        markers = [None] * 17
        
        for j in range(len(self.templates)):
            if self.templates[j] is None:
                continue
            
            try:
                # Scale search region
                t = int((self.template_positions[j][0] - 100) * scale_y)
                l = max(int((self.template_positions[j][1] - 100) * scale_x), 0)
                t_end = int((self.template_positions[j][0] + 100 + self.template_sizes[j][0]) * scale_y)
                l_end = int((self.template_positions[j][1] + 250 + self.template_sizes[j][1]) * scale_x)
                
                search_range = ima[t:t_end, l:l_end]
                
                # Resize template to match scale
                scaled_template = cv2.resize(
                    self.templates[j],
                    (int(self.template_sizes[j][1] * scale_x), 
                     int(self.template_sizes[j][0] * scale_y))
                )
                
                res = cv2.matchTemplate(search_range, scaled_template, cv2.TM_CCOEFF)
                _, max_val, _, max_loc = cv2.minMaxLoc(res)
                
                top_left = max_loc
                markers[j] = np.array((
                    t + top_left[1] + int(self.template_points[j][0] * scale_y),
                    l + top_left[0] + int(self.template_points[j][1] * scale_x)
                ))
            except:
                continue
        
        # Interpolate missing markers
        for i in range(3):
            if markers[5 * i + 3] is not None and markers[5 * i + 2] is not None:
                markers[5 * i + 4] = markers[5 * i + 3] * 2 - markers[5 * i + 2]
        
        if markers[14] is not None and markers[9] is not None:
            markers[16] = ((markers[14] * (284 + 260) - markers[9] * 260) / 284).astype(int)
        
        return markers
    
    @staticmethod
    def lead_info(lead):
        mapping = {
            'I': (0, 1), 'II-subset': (5, 6), 'III': (10, 11),
            'aVR': (1, 2), 'aVL': (6, 7), 'aVF': (11, 12),
            'V1': (2, 3), 'V2': (7, 8), 'V3': (12, 13),
            'V4': (3, 4), 'V5': (8, 9), 'V6': (13, 14),
            'II': (15, 16),
        }
        begin, end = mapping[lead]
        return begin // 5, begin, end


def find_line_by_topdown_sweep(ima):
    """Extract signal boundaries"""
    top = np.argmin(ima, axis=0)
    median_top = int(np.median(top))
    top[top == 0] = median_top
    top[top > median_top + 300] = median_top
    
    strip_width = 64
    for strip_left in range(0, ima.shape[1], strip_width):
        median_top_strip = int(np.median(top[strip_left:strip_left + strip_width]))
        if median_top_strip > median_top + 300:
            median_top_strip = median_top
        
        strip = ima[median_top_strip + 80:, strip_left:strip_left + strip_width]
        all_white = strip.all(axis=1)
        
        if all_white.size > 0:
            first_white_row = np.argmax(all_white)
            if first_white_row > 0 or all_white[0]:
                first_white_row += median_top_strip + 80
                mask = top > first_white_row
                mask[:strip_left] = False
                mask[strip_left + strip_width:] = False
                top[mask] = median_top_strip
    
    mask = np.tile(np.arange(len(ima)).reshape(-1, 1), reps=(1, ima.shape[1]))
    mask = mask >= top
    ima &= mask
    
    bottom = np.argmax(ima, axis=0)
    bottomx = np.maximum(bottom, np.median(top) + 100)
    mask = np.tile(np.arange(len(ima)).reshape(-1, 1), reps=(1, ima.shape[1]))
    mask = mask < bottomx
    ima |= mask
    ima[:, :-1] |= mask[:, 1:]
    ima[:, 1:] |= mask[:, :-1]
    
    return top, bottom

def get_lead_from_top_bottom(tops, bottoms, lead, number_of_rows, markers):
    """Extract lead signal from boundaries"""
    line, begin, end = MarkerFinder.lead_info(lead)
    top = tops[line]
    bottom = bottoms[line]
    begin, end = markers[begin], markers[end]
    
    baseline = np.linspace(begin[0], end[0], end[1] - begin[1])
    pred0 = (top[begin[1]:end[1]] + bottom[begin[1]:end[1]]) / 2
    
    if len(pred0) < len(baseline):
        baseline = baseline[:len(pred0)]
    
    pred = baseline - pred0
    pred /= SCALING_FACTOR
    
    # Remove marker artifacts
    if lead in ['aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']:
        pred[:4] = np.where(pred[:4] > MARKER_ARTIFACT_THRESHOLD, pred[4], pred[:4])
    if lead in ['I', 'II-subset', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3']:
        pred[-5:] = np.where(pred[-5:] > MARKER_ARTIFACT_THRESHOLD, pred[-6], pred[-5:])
    if lead in ['I', 'II-subset', 'III', 'II']:
        pred[:2] = pred[2]
    
    pred = np.interp(np.linspace(0, 1, number_of_rows),
                     np.linspace(0, 1, len(pred)), pred)
    
    # Adaptive outlier removal
    outlier_mask = (pred < OUTLIER_LOW_THRESHOLD) | (pred > OUTLIER_HIGH_THRESHOLD)
    if np.any(outlier_mask):
        for i in np.where(outlier_mask)[0]:
            start_idx = max(0, i - 3)
            end_idx = min(len(pred), i + 4)
            neighbors = pred[start_idx:end_idx]
            valid_neighbors = neighbors[
                (neighbors >= OUTLIER_LOW_THRESHOLD) & 
                (neighbors <= OUTLIER_HIGH_THRESHOLD)
            ]
            if len(valid_neighbors) > 0:
                pred[i] = np.median(valid_neighbors)
            else:
                pred[i] = 0
    
    pred = medfilt(pred, kernel_size=MEDIAN_FILTER_SIZE)
    
    # Tail correction
    if lead == 'II':
        n_tail = number_of_rows // 48
        tail_values = pred[-n_tail:]
        tail_median = np.median(tail_values)
        tail_std = np.std(tail_values)
        threshold = min(OUTLIER_HIGH_THRESHOLD, tail_median + TAIL_CORRECTION_FACTOR * tail_std)
        pred[-n_tail:] = np.where(np.abs(pred[-n_tail:]) <= threshold, pred[-n_tail:], tail_median)
    
    if lead in ['V4', 'V5', 'V6']:
        n_tail = number_of_rows // 12
        tail_values = pred[-n_tail:]
        tail_median = np.median(tail_values)
        tail_std = np.std(tail_values)
        threshold = min(OUTLIER_HIGH_THRESHOLD, tail_median + TAIL_CORRECTION_FACTOR * tail_std)
        pred[-n_tail:] = np.where(np.abs(pred[-n_tail:]) <= threshold, pred[-n_tail:], tail_median)
    
    return pred

def apply_einthoven(preds):
    """Apply Einthoven's law"""
    residual = preds['I'] + preds['III'] - preds['II'][:len(preds['III'])]
    correction = residual / 3
    preds['I'] -= correction
    preds['III'] -= correction
    preds['II'][:len(preds['III'])] += correction
    
    residual = preds['aVR'] + preds['aVL'] + preds['aVF']
    correction = residual / 3
    preds['aVR'] -= correction
    preds['aVL'] -= correction
    preds['aVF'] -= correction
    
    residual = (2 * preds['aVR'] - 2 * preds['aVF'] + 
                3 * preds['II'][len(preds['I']):len(preds['I']) + len(preds['aVR'])])
    correction = residual / 17
    preds['aVR'] -= 2 * correction
    preds['aVF'] += 2 * correction
    preds['II'][len(preds['I']):len(preds['I']) + len(preds['aVR'])] -= 3 * correction

def convert_scanned_color(ima, markers, n_timesteps):
    """Convert image to signals using marker-based extraction"""
    crop_top = int(400 * (ima.shape[0] / 1652))  # Scale crop
    
    # Use red channel and binarize
    ima = ima[crop_top:, :, 2] > IMAGE_BINARIZATION_THRESHOLD
    
    # Morphological smoothing
    iima = ima.astype(np.uint8)
    ima = (iima[:-2, :-2] + iima[:-2, 1:-1] + iima[:-2, 2:] +
           iima[1:-1, :-2] + iima[1:-1, 1:-1] + iima[1:-1, 2:] +
           iima[2:, :-2] + iima[2:, 1:-1] + iima[2:, 2:]) >= 7
    
    tops, bottoms = [], []
    for i in range(4):
        top, bottom = find_line_by_topdown_sweep(ima)
        tops.append(top)
        bottoms.append(bottom)
    
    tops = [t + crop_top for t in tops]
    bottoms = [b + crop_top for b in bottoms]
    
    n_timesteps['II-subset'] = n_timesteps['I']
    preds = {}
    for lead in LEADS + ['II-subset']:
        pred = get_lead_from_top_bottom(tops, bottoms, lead, n_timesteps[lead], markers)
        preds[lead] = pred
    
    preds['II'][:len(preds['II-subset'])] = (
        preds['II'][:len(preds['II-subset'])] + preds['II-subset']
    ) / 2
    del preds['II-subset']
    
    apply_einthoven(preds)
    
    return preds

def is_color_image(ima):
    return ima.std(axis=2).mean() != 0


def main():

    train = pd.read_csv('/kaggle/input/physionet-ecg-image-digitization/train.csv')
    test = pd.read_csv('/kaggle/input/physionet-ecg-image-digitization/test.csv')
    print(f"\n Data: {len(train)} train, {len(test)} test")
    
    mean_dict = fit_mean_model(train)
    
 
    mf = MarkerFinder()
    
    # Validation
    print("\n Validating on holdout set...")
    val_split = int(len(train) * 0.8)
    val_df = train.iloc[val_split:]
    
    snr_list = []
    for idx, row in tqdm(val_df.iterrows(), total=len(val_df), desc="Validating"):
        csv_path = f'{TRAIN_DIR}{row.id}/{row.id}.csv'
        img_path = f'{TRAIN_DIR}{row.id}/{row.id}-0001.png'
        
        if not os.path.exists(csv_path) or not os.path.exists(img_path):
            continue
        
        try:
            ima = cv2.imread(img_path)
            if not is_color_image(ima):
                continue
            
            markers = mf.find_markers(ima)
            if markers is None or sum(1 for m in markers if m is not None) < 10:
                continue
            
            n_timesteps = {lead: row.fs * 10 if lead == 'II' else row.fs * 10 // 4 for lead in LEADS}
            preds = convert_scanned_color(ima, markers, n_timesteps)
            
            labels = pd.read_csv(csv_path)
            sum_signal = 0
            sum_noise = 0
            
            for lead in labels.columns:
                label = labels[lead].dropna().values
                if len(label) == 0:
                    continue
                
                pred = preds[lead][:len(label)]
                aligned_pred = align_signals(label, pred, int(row.fs * MAX_TIME_SHIFT))
                p_signal, p_noise = compute_power(label, aligned_pred)
                sum_signal += p_signal
                sum_noise += p_noise
            
            snr = compute_snr(sum_signal, sum_noise)
            snr_list.append(snr)
        except:
            continue
    
    if snr_list:
        mean_snr = np.mean(snr_list)
        val_score = max(10 * np.log10(mean_snr), -PERFECT_SCORE)
        print(f"\n✓ Validated on {len(snr_list)} samples")
        print(f"✓ Mean SNR: {mean_snr:.4f}")
        print(f"✓ Validation Score: {val_score:.2f} dB\n")
    else:
        print("\n No successful validations\n")
    
    print("\n Processing...")
    submission_data = []
    old_id = None
    processed = 0
    fallback = 0
    
    for idx, row in tqdm(test.iterrows(), total=len(test), desc="Extracting"):
        if row.id != old_id:
            path = f"{TEST_DIR}{row.id}.png"
            ima = cv2.imread(path)
            
            if is_color_image(ima):
                try:
                    markers = mf.find_markers(ima)
                    if markers is not None and sum(1 for m in markers if m is not None) >= 10:
                        n_timesteps = {lead: row.fs * 10 if lead == 'II' else row.fs * 10 // 4 for lead in LEADS}
                        preds = convert_scanned_color(ima, markers, n_timesteps)
                        processed += 1
                    else:
                        preds = None
                        fallback += 1
                except:
                    preds = None
                    fallback += 1
            else:
                preds = None
                fallback += 1
            
            old_id = row.id
        
        if preds is not None:
            pred = preds[row.lead]
        else:
            pred = mean_dict[row.lead].mean(axis=0)
            pred = np.interp(np.linspace(0, 1, row.number_of_rows),
                           np.linspace(0, 1, len(pred)), pred)
        
        for timestep in range(row.number_of_rows):
            signal_id = f"{row.id}_{timestep}_{row.lead}"
            submission_data.append({'id': signal_id, 'value': float(pred[timestep])})
    
    print(f"\n Processed: {processed} | Fallback: {fallback}")
    
    submission = pd.DataFrame(submission_data)
    submission.to_csv('submission.csv', index=False)
    print(f" Saved: {len(submission):,} predictions")
    print(f"   Range: [{submission['value'].min():.4f}, {submission['value'].max():.4f}]")
   

if __name__ == "__main__":
    main()